In [1]:
%load_ext autoreload
%autoreload 2
    
from pathlib import Path
import metapack as mp
import geopandas as gpd
import pandas as pd
from auto_tqdm import tqdm 

doc = mp.jupyter.open_source_package()
doc.set_sys_path()
import pylib

ea_epsg = 2163 #US Equal Area projection

import logging
logging.basicConfig()

from pylib import lines_logger, points_logger
lines_logger.setLevel(logging.DEBUG)
points_logger.setLevel(logging.DEBUG)

pkg_root = Path(doc.path).parent
pkg = mp.open_package(pkg_root)
pkg

# Open Street Map Extract for Demographic Search
`civicknowledge.com-osm-demosearch-1.1.2` Last Update: 2021-02-03T01:33:47

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [geohash_tags](data/geohash_tags.csv)**. Points converted to counts of tags per geohash
* ** [residential_roads](data/residential_roads.csv)**. Residential roads per 4 digit geohash
* ** [nonres_roads](data/nonres_roads.csv)**. Non residential roads per 4 digit geohash

## References
<ul><li> <strong>us_geohashes</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_geohashes</em>. All 4 digit geohases in the continential US</li><li> <strong><a href="https://download.geofabrik.de/north-america-latest.osm.pbf">north-america-latest</a></strong>. OSM North America extract</li><li> <strong>points</string>, <em>data/csv/points.csv</em>. Points from the OSM file</li><li> <strong>lines</string>, <em>data/csv/lines.csv</em>. Lines from the OSM file</li><li> <strong>multipolygons</string>, <em>data/csv/multipolygons.csv</em>. Polygons from the OSM file</li><li> <strong>multilinestrings</string>, <em>data/csv/multilinestrings.csv</em>. Lines from the OSM file</li><li> <strong>other_relations</string>, <em>data/csv/other_relations.csv</em>. Other geo data from the OSM file</li><ul>

In [15]:

pylib.build_lines(pkg)


INFO:pylib.lines:Split the input file
INFO:pylib.lines:   5328 splits keys
INFO:pylib.lines:Run the overlay process


Overlay Geohash:   0%|          | 0/5328 [00:00<?, ?it/s]

INFO:pylib.lines:Simplify lines
INFO:pylib.lines:   43558 simplified keys
INFO:pylib.lines:Write the roads files


   5300 recombine keys


In [11]:
t = pkg.resource('nonres_roads').dataframe()

In [12]:
t

,geohash,highway,geometry
0,c287,2,"LINESTRING (477089 5368078, 476883 5368125, 47..."
1,c287,2,"LINESTRING (477998 5367584, 477301 5368193)"
2,c287,2,"LINESTRING (472861 5365087, 472893 5365089)"
3,c287,2,"LINESTRING (473835 5372347, 473823 5372322)"
4,c287,2,"LINESTRING (471390 5370525, 471156 5369915)"
...,...,...,...
2501664,dnky,t,"LINESTRING (295020 4017025, 294985 4016769, 29..."
2501665,dr5r,3,"LINESTRING (593907 4513326, 593922 4513277)"
2501666,dr5r,1,"LINESTRING (591242 4495881, 591244 4495878)"
2501667,dr5r,1,"LINESTRING (590298 4496967, 590259 4497012)"


In [14]:
pylib.build_points(pkg)

INFO:pylib.points:Make tags dataframe
INFO:pylib.points:Extract class Columns
INFO:pylib.points:Make geotags dataframe
